In [1]:
from os import listdir
from os.path import isfile, join
import numpy as np
import pandas as pd
import cv2
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
%matplotlib inline
#C:/Users/Shubham/Documents/Image_Processing/Dataset/Train/color/

In [2]:
mypath='/Dataset/Train/color/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images = []
Y = []
for n in range(0,len(onlyfiles)):
    Full_size = cv2.imread( join(mypath,onlyfiles[n]))
    r,c=Full_size.shape[:2]
    out_r= 120
    resized = cv2.resize(Full_size, (128,128), interpolation=cv2.INTER_CUBIC)
    images.append(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    Y.append(1)

In [3]:
mypath='/Dataset/Train/grey/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
images1 = []

for n in range(0,len(onlyfiles)):
    Full_size = cv2.imread( join(mypath,onlyfiles[n]))
    r,c=Full_size.shape[:2]
    out_r= 120
    resized = cv2.resize(Full_size, (128,128), interpolation=cv2.INTER_CUBIC)
    images1.append(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    Y.append(0)

In [4]:
X = images + images1

In [12]:
mypath='/Dataset/Test/color/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
test = []
y = []
for n in range(0,len(onlyfiles)):
    Full_size = cv2.imread( join(mypath,onlyfiles[n]))
    r,c=Full_size.shape[:2]
    out_r= 120
    resized = cv2.resize(Full_size, (128,128), interpolation=cv2.INTER_CUBIC)
    test.append(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    y.append(1)

In [13]:
mypath='/Dataset/Test/grey/'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
test1 = []
for n in range(0,len(onlyfiles)):
    Full_size = cv2.imread( join(mypath,onlyfiles[n]))
    r,c=Full_size.shape[:2]
    out_r= 120
    resized = cv2.resize(Full_size, (128,128), interpolation=cv2.INTER_CUBIC)
    test1.append(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    y.append(0)


In [ ]:
mypath='/Use Case 1/Challenging Images/color'
onlyfiles = [ f for f in listdir(mypath) if isfile(join(mypath,f)) ]
sp = []
d = []
for n in range(0,len(onlyfiles)):
    Full_size = cv2.imread( join(mypath,onlyfiles[n]))
    r,c=Full_size.shape[:2]
    out_r= 120
    resized = cv2.resize(Full_size, (128,128), interpolation=cv2.INTER_CUBIC)
    sp.append(cv2.cvtColor(resized, cv2.COLOR_BGR2RGB))
    d.append(1)


In [14]:
x = test + test1 + X
y = y + Y

In [15]:
df = pd.DataFrame()
df["labels"]=y
df["images"]=x



In [16]:
print(len(df), df.images[2].shape)


256 (128, 128, 3)


In [18]:
np.savez("x_images_arrays", x)
np.savez("y_infiltration_labels", y)

In [19]:
# Load npz file containing image arrays
x_npz = np.load("x_images_arrays.npz")
g = x_npz['arr_0']
# Load binary encoded labels for Lung Infiltrations: 0=Not_infiltration 1=Infiltration
y_npz = np.load("y_infiltration_labels.npz")
h = y_npz['arr_0']

In [40]:
# First split the data in two sets, 80% for training, 20% for Val/Test)
from sklearn.model_selection import train_test_split

X_train, X_valtest, y_train, y_valtest = train_test_split(g,h, test_size=0.2, stratify=h)

# Second split the 20% into validation and test sets
X_test, X_val, y_test, y_val = train_test_split(X_valtest, y_valtest, test_size=0.5, stratify=y_valtest)

In [41]:
print(np.array(X_train).shape)
print(np.array(X_val).shape)
print(np.array(X_test).shape)

(204, 128, 128, 3)
(26, 128, 128, 3)
(26, 128, 128, 3)


In [42]:
from keras.preprocessing.image import ImageDataGenerator
from keras import layers, models, optimizers
from keras import backend as K

In [43]:
K.image_data_format()


'channels_last'

In [44]:
img_width, img_height = 128, 128
nb_train_samples = len(X_train)
nb_validation_samples = len(X_val)
epochs = 4
batch_size = 8

In [68]:
model = models.Sequential()

model.add(layers.Conv2D(32, (3, 3), input_shape=(img_width, img_height, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation("sigmoid"))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(64, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation("sigmoid"))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Conv2D(128, (3, 3)))
model.add(layers.BatchNormalization())
model.add(layers.Activation("relu"))

model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64))
model.add(layers.BatchNormalization())
model.add(layers.Activation("sigmoid"))

model.add(layers.Dense(1))
model.add(layers.BatchNormalization())
model.add(layers.Activation("softmax"))

model.compile(
    loss='binary_crossentropy',
    optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0),
    metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 126, 126, 32)      896       
_________________________________________________________________
batch_normalization_26 (Batc (None, 126, 126, 32)      128       
_________________________________________________________________
activation_26 (Activation)   (None, 126, 126, 32)      0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 61, 61, 64)        18496     
_________________________________________________________________
batch_normalization_27 (Batc (None, 61, 61, 64)        256       
_________________________________________________________________
activation_27 (Activation)   (None, 61, 61, 64)        0         
__________

In [69]:
train_datagen = ImageDataGenerator(rescale=1. / 255, horizontal_flip=True, rotation_range=30)
valtest_datagen = ImageDataGenerator(rescale=1. / 255)

In [72]:
train_generator = train_datagen.flow(np.array(X_train), y_train, batch_size=batch_size)
validation_generator = valtest_datagen.flow(np.array(X_val), y_val, batch_size=batch_size)
test_generator = valtest_datagen.flow(np.array(X_test), y_test, batch_size=batch_size)

In [74]:
history = model.fit_generator(
    train_generator, 
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size
)

model.save_weights('weights.h5')

Epoch 1/4
25/25 [==============================] - 68s 3s/step - loss: 7.8116 - acc: 0.5100 - val_loss: 7.3069 - val_acc: 0.5417
Epoch 2/4
25/25 [==============================] - 51s 2s/step - loss: 7.6521 - acc: 0.5200 - val_loss: 7.3069 - val_acc: 0.5417
Epoch 3/4
25/25 [==============================] - 50s 2s/step - loss: 7.5754 - acc: 0.5248 - val_loss: 7.3069 - val_acc: 0.5417
Epoch 4/4
25/25 [==============================] - 51s 2s/step - loss: 8.0541 - acc: 0.4948 - val_loss: 7.3069 - val_acc: 0.5417
